In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib notebook

import chardet
from bs4 import UnicodeDammit

# Считывание данных в формате .csv в DataFrame

In [67]:
data_encodings = []

with open("Data\credit_train.csv", 'rb') as f:
    rawdata = b''.join([f.readline() for _ in range(1000)])
    data_encodings.append(chardet.detect(rawdata)['encoding'])
    data_encodings.append(UnicodeDammit(rawdata).original_encoding)
       
print(f"Кодировка исходного файла: {data_encodings}")

Кодировка исходного файла: ['KOI8-R', 'koi8-r']


Однако, кодировка KOI8-R не подходит для корректного открытия файла, поэтому было решено проверить наиболее распространенные кодировки для файлов, содержащих кириллицу.

In [69]:
data_encodings.extend(['utf-16','CP866', 'Windows-1251', 'utf-8', 'cp1251'])

In [72]:
file_encoding = ''
for encoding in data_encodings:
    try:
        with pd.read_csv("Data\credit_train.csv", encoding=encoding, sep=";", index_col='client_id', iterator=True) as reader:
            print('--------------------------------')
            print(f'Кодировка: {encoding}')
            print(reader.get_chunk(5))
            print('--------------------------------\n')
        if input("Если кодировка верна, введи 'верно' >>>>  ") == 'верно':
            file_encoding = encoding
    except UnicodeDecodeError:
        print(f'Кодировка: {encoding} не подходит!')

--------------------------------
Кодировка: KOI8-R
          gender   age marital_status job_position credit_sum  credit_month  \
client_id                                                                     
1              M   NaN            NaN          UMN   59998,00            10   
2              F   NaN            MAR          UMN   10889,00             6   
3              M  32.0            MAR          SPC   10728,00            12   
4              F  27.0            NaN          SPC   12009,09            12   
5              M  45.0            NaN          SPC        NaN            10   

           tariff_id score_shk education        living_region  monthly_income  \
client_id                                                                       
1                1.6       NaN       GRD   йпюямндюпяйхи йпюи         30000.0   
2                1.1       NaN       NaN               лняйбю             NaN   
3                1.1       NaN       NaN      нак яюпюрнбяйюъ          

Если кодировка верна, введи 'верно'верно


In [82]:
init_data = pd.read_csv("Data\credit_train.csv", encoding=file_encoding, sep=";")
init_data.head(5)

,client_id,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
0,1,M,NaN,NaN,UMN,"59998,00",10,1.6,NaN,GRD,КРАСНОДАРСКИЙ КРАЙ,30000.0,1.0,1.0,0
1,2,F,NaN,MAR,UMN,"10889,00",6,1.1,NaN,NaN,МОСКВА,NaN,2.0,0.0,0
2,3,M,32.0,MAR,SPC,"10728,00",12,1.1,NaN,NaN,ОБЛ САРАТОВСКАЯ,NaN,5.0,0.0,0
3,4,F,27.0,NaN,SPC,"12009,09",12,1.1,NaN,NaN,ОБЛ ВОЛГОГРАДСКАЯ,NaN,2.0,0.0,0
4,5,M,45.0,NaN,SPC,NaN,10,1.1,"0,421385",SCH,ЧЕЛЯБИНСКАЯ ОБЛАСТЬ,NaN,1.0,0.0,0


In [83]:
print(f'Число наблюдений - {init_data.shape[0]}')
print(f'Число фичей (переменных) - {init_data.shape[1]}')

Число наблюдений - 170746
Число фичей (переменных) - 15


# Предварительная подготовка данных

## Удаление очевидных бесполезных переменных

В данном случае имеются ввиду переменные, у которых количество категорий совпадает с количеством наблюдений, или, наоборот, переменные с одним уникальным значением.

In [84]:
init_data.nunique()

client_id               170746
gender                       2
age                         54
marital_status               5
job_position                18
credit_sum               42769
credit_month                31
tariff_id                   32
score_shk                16279
education                    5
living_region              301
monthly_income            1591
credit_count                21
overdue_credit_count         4
open_account_flg             2
dtype: int64

Откинем столбец `client_id` поскольку он совпадает с номером наблюдения и не несет никакой предсказательной способности.

In [85]:
init_data.drop('client_id', inplace=True, axis=1)

In [86]:
init_data.head(5)

,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
0,M,NaN,NaN,UMN,"59998,00",10,1.6,NaN,GRD,КРАСНОДАРСКИЙ КРАЙ,30000.0,1.0,1.0,0
1,F,NaN,MAR,UMN,"10889,00",6,1.1,NaN,NaN,МОСКВА,NaN,2.0,0.0,0
2,M,32.0,MAR,SPC,"10728,00",12,1.1,NaN,NaN,ОБЛ САРАТОВСКАЯ,NaN,5.0,0.0,0
3,F,27.0,NaN,SPC,"12009,09",12,1.1,NaN,NaN,ОБЛ ВОЛГОГРАДСКАЯ,NaN,2.0,0.0,0
4,M,45.0,NaN,SPC,NaN,10,1.1,"0,421385",SCH,ЧЕЛЯБИНСКАЯ ОБЛАСТЬ,NaN,1.0,0.0,0


## Преобразование типов данных

В данном разделе подразумевается преобразование типов данных к корректным, например, `credit_sum` в датасете имеет тип `object`, чего не должно быть. Также, часто переменные-флаги, такие как "наличие оттока", "наличие дефолта" и т.п. записываются как целочисленные значения.

In [89]:
init_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170746 entries, 0 to 170745
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   gender                170746 non-null  object 
 1   age                   170743 non-null  float64
 2   marital_status        170743 non-null  object 
 3   job_position          170746 non-null  object 
 4   credit_sum            170744 non-null  object 
 5   credit_month          170746 non-null  int64  
 6   tariff_id             170746 non-null  float64
 7   score_shk             170739 non-null  object 
 8   education             170741 non-null  object 
 9   living_region         170554 non-null  object 
 10  monthly_income        170741 non-null  float64
 11  credit_count          161516 non-null  float64
 12  overdue_credit_count  161516 non-null  float64
 13  open_account_flg      170746 non-null  int64  
dtypes: float64(5), int64(2), object(7)
memory usage: 18.

In [92]:
for column in ['score_shk', 'credit_sum']:
    init_data[column] = init_data[column].str.replace(',', '.').astype('float')

In [94]:
for column in ['tariff_id', 'open_account_flg']:
    init_data[column] = init_data[column].astype('object')

In [95]:
init_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170746 entries, 0 to 170745
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   gender                170746 non-null  object 
 1   age                   170743 non-null  float64
 2   marital_status        170743 non-null  object 
 3   job_position          170746 non-null  object 
 4   credit_sum            170744 non-null  float64
 5   credit_month          170746 non-null  int64  
 6   tariff_id             170746 non-null  object 
 7   score_shk             170739 non-null  float64
 8   education             170741 non-null  object 
 9   living_region         170554 non-null  object 
 10  monthly_income        170741 non-null  float64
 11  credit_count          161516 non-null  float64
 12  overdue_credit_count  161516 non-null  float64
 13  open_account_flg      170746 non-null  object 
dtypes: float64(6), int64(1), object(7)
memory usage: 18.

In [96]:
init_data.head(5)

,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
0,M,NaN,NaN,UMN,59998.00,10,1.6,NaN,GRD,КРАСНОДАРСКИЙ КРАЙ,30000.0,1.0,1.0,0
1,F,NaN,MAR,UMN,10889.00,6,1.1,NaN,NaN,МОСКВА,NaN,2.0,0.0,0
2,M,32.0,MAR,SPC,10728.00,12,1.1,NaN,NaN,ОБЛ САРАТОВСКАЯ,NaN,5.0,0.0,0
3,F,27.0,NaN,SPC,12009.09,12,1.1,NaN,NaN,ОБЛ ВОЛГОГРАДСКАЯ,NaN,2.0,0.0,0
4,M,45.0,NaN,SPC,NaN,10,1.1,0.421385,SCH,ЧЕЛЯБИНСКАЯ ОБЛАСТЬ,NaN,1.0,0.0,0
